In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .map(maskL8sr)
Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

# JRC水体频率，2000-2020

In [8]:
# dataset = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filter(ee.Filter.gte('year',2000))
# waterPixel = dataset.map(lambda i : i.clip(roi).eq(2)).sum()
# validPixel = dataset.map(lambda i : i.clip(roi).gt(0)).sum()
# Map.addLayer(waterPixel, {'palette':['white','red'],'min':0,'max':85}, 'waterPixel')
# Map.addLayer(validPixel, {'palette':['white','red'],'min':0,'max':85}, 'validPixel')
# waterFrequency = waterPixel.divide(validPixel).multiply(100)
# Map.addLayer(waterFrequency, {'palette':['white','blue'],'min':0,'max':100}, 'waterFrequency')

In [5]:
dataset = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filter(ee.Filter.gte('year',2000))
waterPixel = dataset.map(lambda i : i.clip(roi).eq(2)).sum()
validPixel = dataset.map(lambda i : i.clip(roi).gt(0)).sum()
waterFrequency = waterPixel.divide(validPixel).multiply(100)
# Map.addLayer(waterFrequency, {'palette':['white','blue'],'min':0,'max':1}, 'waterFrequency')

In [ ]:
minmax = waterPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
    'tileScale': 2,
})
print(minmax.getInfo())

In [ ]:
minmax = validPixel.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'tileScale': 2,
})
print(minmax.getInfo())

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
%pylab inline
region = [113.6, 29.9, 115.1, 31.4]
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterFrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'JRC/2014-2020)', fontsize=20)